####  RNN - Sentiment Analysis  based on Keras

Here, not like the language model, each sequence has only one label. It is a sequence classifciation model


In [1]:
from tensorflow import keras
sequence = keras.preprocessing.sequence
Sequential = keras.models.Sequential
Dense = keras.layers.Dense
Embedding = keras.layers.Embedding
LSTM = keras.layers.LSTM
imdb = keras.datasets.imdb

In [2]:
max_features = 20000
maxlen = 80
batch_size = 128

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


#### sub sample date

In [3]:
x_train = x_train[:2000]
y_train = y_train[:2000]

In [6]:
y_train[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [7]:
x_test = x_test[:500]
y_test = y_test[:500]

In [8]:
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

2000 train sequences
500 test sequences
Pad sequences (samples x time)
x_train shape: (2000, 80)
x_test shape: (500, 80)


### Keras Sequential model to build the model

In [9]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 50))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test accuracy:', acc)

Build model...
Train...


/anaconda3/envs/week3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2000 samples, validate on 500 samples
Epoch 1/3
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6926 - acc: 0.5085 - val_loss: 0.6915 - val_acc: 0.4960
Epoch 2/3
2000/2000 [==============================] - 5s 2ms/step - loss: 0.6716 - acc: 0.6090 - val_loss: 0.6388 - val_acc: 0.6560
Epoch 3/3
500/500 [==============================] - 0s 559us/step
Test accuracy: 0.7659999976158142


### Keras Model class used with functional API

1. define input and target layer
2. call models to set input and target
3. layers are used to connect between input and target layers

In [13]:
Model = keras.models.Model
Input = keras.layers.Input
Dense = keras.layers.Dense

In [14]:
input_a = Input(shape=(maxlen,))
embed = Embedding(max_features, 50)(input_a)
hidden_layer = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(embed)
target = Dense(1, activation='sigmoid')(hidden_layer)
model = Model(inputs=input_a, outputs=target)

In [15]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test accuracy:', acc)

Train...


/anaconda3/envs/week3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2000 samples, validate on 500 samples
Epoch 1/3
2000/2000 [==============================] - 14s 7ms/step - loss: 0.6852 - acc: 0.5745 - val_loss: 0.6307 - val_acc: 0.7220
Epoch 2/3
2000/2000 [==============================] - 12s 6ms/step - loss: 0.5803 - acc: 0.7315 - val_loss: 0.6181 - val_acc: 0.6520
Epoch 3/3
500/500 [==============================] - 0s 862us/step
Test score: 0.5086510560512543
Test accuracy: 0.764


###  stacked LSTM

In [16]:
input_a = Input(shape=(maxlen,))
embed = Embedding(max_features, 50)(input_a)
hidden_layer_one = LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(embed)
hidden_layer_two = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(hidden_layer_one)
target = Dense(1, activation='sigmoid')(hidden_layer_two)
model = Model(inputs=input_a, outputs=target)

In [17]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=5,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test accuracy:', acc)

Train...


/anaconda3/envs/week3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2000 samples, validate on 500 samples
Epoch 1/5
2000/2000 [==============================] - 28s 14ms/step - loss: 0.6867 - acc: 0.5445 - val_loss: 0.6297 - val_acc: 0.6740
Epoch 2/5
2000/2000 [==============================] - 28s 14ms/step - loss: 0.4544 - acc: 0.7950 - val_loss: 0.5230 - val_acc: 0.7580
Epoch 3/5
2000/2000 [==============================] - 23s 12ms/step - loss: 0.1629 - acc: 0.9420 - val_loss: 0.5969 - val_acc: 0.7940
Epoch 4/5
2000/2000 [==============================] - 22s 11ms/step - loss: 0.0645 - acc: 0.9785 - val_loss: 0.8957 - val_acc: 0.7380
Epoch 5/5
500/500 [==============================] - 1s 2ms/step
Test score: 0.947280821800232
Test accuracy: 0.7579999990463256


### bidrectional LSTM
```
keras.layers.Bidirectional(layer, merge_mode='concat', weights=None)
```
layer: Recurrent instance.
merge_mode: Mode by which outputs of the forward and backward RNNs will be combined. One of {'sum', 'mul', 'concat', 'ave', None}. If None, the outputs will not be combined, they will be returned as a list.

In [16]:
Bidirectional = keras.layers.Bidirectional
input_a = Input(shape=(maxlen,))
embed = Embedding(max_features, 50)(input_a)
hidden_layer_one = LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(embed)
hidden_layer_two = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2))(hidden_layer_one)
target = Dense(1, activation='sigmoid')(hidden_layer_two)
model = Model(inputs=input_a, outputs=target)

In [17]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...


/anaconda3/envs/week3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2000 samples, validate on 500 samples
Epoch 1/5
2000/2000 [==============================] - 37s 19ms/step - loss: 0.6832 - acc: 0.5385 - val_loss: 0.6172 - val_acc: 0.6760
Epoch 2/5
2000/2000 [==============================] - 36s 18ms/step - loss: 0.4333 - acc: 0.8065 - val_loss: 0.4552 - val_acc: 0.7920
Epoch 3/5
2000/2000 [==============================] - 34s 17ms/step - loss: 0.1874 - acc: 0.9275 - val_loss: 0.5386 - val_acc: 0.7520
Epoch 4/5
2000/2000 [==============================] - 34s 17ms/step - loss: 0.0740 - acc: 0.9755 - val_loss: 0.7045 - val_acc: 0.7660
Epoch 5/5
500/500 [==============================] - 1s 3ms/step
Test score: 0.8478314962387085
Test accuracy: 0.7640000004768371


###  LSTM hiddent average vector

In [18]:
batch_size = 128
K = keras.backend
Lambda = keras.layers.Lambda
input_a = Input(shape=(maxlen,))
embed = Embedding(max_features, 50)(input_a)
hidden_layer_one = LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(embed)
hidden_layer_two = LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(hidden_layer_one)
final_vector = Lambda(lambda x: K.mean(x, axis=1), output_shape=(128,))(hidden_layer_two)
target = Dense(1, activation='sigmoid')(final_vector)
model = Model(inputs=input_a, outputs=target)

In [19]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)

print('Test accuracy:', acc)

Train...


/anaconda3/envs/week3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2000 samples, validate on 500 samples
Epoch 1/3
2000/2000 [==============================] - 16s 8ms/step - loss: 0.6926 - acc: 0.5225 - val_loss: 0.6941 - val_acc: 0.4760
Epoch 2/3
2000/2000 [==============================] - 11s 5ms/step - loss: 0.6382 - acc: 0.6130 - val_loss: 0.5868 - val_acc: 0.7160
Epoch 3/3
500/500 [==============================] - 1s 1ms/step
Test score: 0.5769503536224365
Test accuracy: 0.7180000052452088


###  LSTM concatenate

In [20]:
K = keras.backend
Lambda = keras.layers.Lambda
input_a = Input(shape=(maxlen,))
embed = Embedding(max_features, 50)(input_a)
hidden_layer_one = LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(embed)
hidden_layer_two = LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(hidden_layer_one)
final_vector1 = Lambda(lambda x: K.mean(x, axis=1), output_shape=(128,))(hidden_layer_two)
final_vector2 = Lambda(lambda x: K.mean(x, axis=1), output_shape=(128,))(hidden_layer_one)
final_vector = keras.layers.Average()([final_vector1, final_vector2])
target = Dense(1, activation='sigmoid')(final_vector)
model = Model(inputs=input_a, outputs=target)

In [21]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test accuracy:', acc)

Train...


/anaconda3/envs/week3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2000 samples, validate on 500 samples
Epoch 1/3
2000/2000 [==============================] - 16s 8ms/step - loss: 0.6932 - acc: 0.4965 - val_loss: 0.6930 - val_acc: 0.4760
Epoch 2/3
2000/2000 [==============================] - 13s 6ms/step - loss: 0.6563 - acc: 0.5895 - val_loss: 0.6205 - val_acc: 0.6520
Epoch 3/3
500/500 [==============================] - 1s 1ms/step
Test score: 0.5880889921188355
Test accuracy: 0.7140000004768372
